## 数据增强 Data Argument

整理目前使用的几种数据增强方法  

1. 传统方法
   增删换改
2. 人工标注
3. 回译

### EDA

In [2]:
import eda

sentence = "精准营销的定义是指在充分了解顾客信息的基础上，针对客户喜好，有针对性地进行产品营销，\
            在掌握一定的顾客信息和市场信息后，将直复营销与数据库营销结合起来的营销新趋势。"
eda.eda(
    sentence,
    alpha_sr=0.1,    # synonym replacement
    alpha_ri=0.1,    # random insertion
    alpha_rs=0.1,    # random swap
    p_rd=0.1,        # random deletion
    num_aug=3,
)

['精准 推销 的 定义 是 指 在 充分 了解 顾客 信息 的 基础上 ， 针对 客户 喜好 ， 有 系统性 地 同步进行 产品 推销 ， 在 掌握 一定 的 顾客 信息 和 市场 信息 后 ， 将 直复 推销 与 数据库 推销 融合 起来 的 推销 新趋势 。',
 '精准 顾客 的 定义 是 指 在 充分 了解 营销 信息 的 基础上 ， 针对 顾客 喜好 ， 有 客户 地 进行 产品 营销 ， 在 掌握 一定 的 针对性 信息 和 市场 信息 营销 ， 将 直复 营销 与 数据库 后 结合 起来 的 营销 新趋势 。',
 '精准 营销 的 定义 是 指 在 充分 了解 顾客 信息 的 基础上 度量 ， 针对 同义 客户 喜好 ， 有 针对性 地 进行 产品 营销 ， 在 掌握 一定 信息 的 顾客 信息 和 市场 信息 后 ， 将 直复 营销 与 数据库 营销 应有 结合 起来 的 营销 新趋势 。',
 '精准 营销 的 定义 是 指 在 充分 了解 顾客 信息 的 基础上 ， 针对 客户 喜好 ， 有 针对性 地 进行 产品 营销 ，                         在 掌握 一定 的 顾客 信息 和 市场 信息 后 ， 将 直复 营销 与 数据库 营销 结合 起来 的 营销 新趋势 。']

### 人工标注

In [1]:
from pigeonXT import annotate
import pandas as pd

df = pd.DataFrame([
    {'title': 'Star wars'},
    {'title': 'The Positively True Adventures of the Alleged Texas Cheerleader-Murdering Mom'},
    {'title': 'Eternal Sunshine of the Spotless Mind'},
    {'title': 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb'},
    {'title': 'Killer klowns from outer space'},
])

labels = ['Adventure', 'Romance', 'Fantasy', 'Science fiction', 'Horror', 'Thriller']

annotations = annotate(df.title,
                      options=labels,
                      task_type='multilabel-classification',
                      buttons_in_a_row=3,
                      reset_buttons_after_click=True,
                      include_skip=True)

HTML(value='0 examples annotated, 6 examples left')

Output()

### Backtranslation

In [2]:
from transformers import MarianMTModel, MarianTokenizer

# Load model
forward_model_name = '../model/Opus-MT/zho-eng'
forward_tokenizer = MarianTokenizer.from_pretrained(forward_model_name)
forward_model = MarianMTModel.from_pretrained(forward_model_name)

backward_model_name = '../model/Opus-MT/eng-zho'
backward_tokenizer = MarianTokenizer.from_pretrained(backward_model_name)  
backward_model = MarianMTModel.from_pretrained(backward_model_name)

stdbuf was not found; communication with perl may hang due to stdio buffering.
stdbuf was not found; communication with perl may hang due to stdio buffering.


In [3]:
# input sentences
src_text = ['我没我也没有说不还了，我也没有说不还了，你要我去打你，也打过来呀，对吧',
            '一旦如果以恶意欠款兴趣进行比较法务之后承担的后果和罚金会更多']

# backtranslation
translated = forward_model.generate(**forward_tokenizer.prepare_translation_batch(src_text))
translated_text = [forward_tokenizer.decode(t, skip_special_tokens=True) for t in translated]

translated = backward_model.generate(**backward_tokenizer.prepare_translation_batch(translated_text))
reversed_text = [backward_tokenizer.decode(t, skip_special_tokens=True) for t in translated]

# print
print("Original Text: {}".format(src_text))
print("Translated Text: {}".format(translated_text))
print("Reversed Text: {}".format(reversed_text))

Original Text: ['我没我也没有说不还了，我也没有说不还了，你要我去打你，也打过来呀，对吧', '一旦如果以恶意欠款兴趣进行比较法务之后承担的后果和罚金会更多']
Translated Text: ["I didn't say no more. I didn't say no more. You wanted me to hit you.", 'The consequences and penalties would be greater if the law were compared with the interest in malignant arrears.']
Reversed Text: ['我没说没有了 我没说没有了 你想让我揍你', '如果将法律与恶性拖欠利息进行比较,后果和惩罚将更大。']
